# Named entity recognition - clinical concepts

In this practical, we will try to build a named entity recognition classifier using spaCy and crfsuite.

Named entity recognition is a structured learning problem, i.e., we want to learn sequence patterns.

We will use data from mtsamples again, and build classifiers that find clinical concepts. The gold standard data is not manually annotated, it is the output of a clinical concept recognition system developed by Zeljko Kraljevic called 'CAT'. This system matches concepts to the entire UMLS. We will only use a few example concepts here.

Part of this material is adapted, inspired etc from:

https://spacy.io/usage/training,

https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html,

https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da

Written by Sumithra Velupillai, March 2019 - acknowledgements and many thanks to Zeljko for the data preparations!

In [3]:
import spacy
from spacy import displacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

import json
import requests

try:
    import sklearn_crfsuite
except ImportError as e:
    !pip install sklearn_crfsuite
    import sklearn_crfsuite

from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy

import random


# 1: corpus
We have prepared training and test data in a json format.

In [5]:
data_url = 'https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_trainingdata_CAT.json?raw=true'
r = requests.get(data_url)
train_data = r.json()

Let's take a look at a random document and its annotations. The json format contains the text itself, and then the start and stop offsets for each entity. What are the instances we want to learn?

In [6]:
train_data[14][1]

{'entities': [[419, 428, 'ANATOMY'],
  [457, 473, 'ANATOMY'],
  [479, 490, 'ANATOMY'],
  [597, 605, 'ANATOMY'],
  [607, 621, 'ANATOMY'],
  [634, 643, 'ANATOMY']]}

# 2: Training a named entity model with spaCy
We can use spaCy to train our own named entity recognition model using their training algorithm.
First we need to load a spaCy English language model, so that we can sentence- and word tokenize.

In [7]:
nlp = spacy.load('en')

What nlp preprocessing parts does this model contain?

In [8]:
nlp.pipe_names

['tagger', 'parser', 'ner']

We have our own named entities that we want to develop a model for. Let's add these entity labels to the spaCy ner pipe.

In [9]:
if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
else:
        ner = nlp.get_pipe("ner")

        
labels = set()
for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            labels.add(ent[2])

We don't want to retrain the other pipeline steps, so let's keep those.

In [10]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
other_pipes = ["tagger", "parser"]
other_pipes

['tagger', 'parser']

What entities do we have?

In [11]:
print(labels)

{'DISEASESYNDROME', 'SIGNSYMPTOM', 'ANATOMY'}


Now let's train our clinical concept ner model. Let's set the number of training iterations.

In [12]:
n_iter=(10)

Now let's train the model.

In [ ]:
with nlp.disable_pipes(*other_pipes):  # only train NER
#with nlp.disable_pipes(other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = spacy.util.minibatch(train_data, size=spacy.util.compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

Losses {'ner': 1987.194958943256}


We have now added a clinical concept entity recognizer in the spaCy nlp model! Let's look at an example document and the predicted entities from the new model.

In [ ]:
text, _ = train_data[6]

In [ ]:
doc2 = nlp(text)
colors = {'ANATOMY': 'lightyellow',
           'DISEASESYNDROME': 'pink',  
           'SIGNSYMPTOM': 'lightgreen'}
displacy.render(doc2, style='ent', jupyter=True, options={'colors':colors})

We can also look at the underlying representation - let's look at one sentence in this document.

In [ ]:
print([(x, x.ent_iob_, x.ent_type_) for x in list(doc2.sents)[5]])

What do you think? Does it seem like the model works well on this document? Are there concepts that are missed? 


# 3: Evaluation
How do we know how good this model is? Let's compare with the 'gold standard' test data.

In [ ]:
scorer = Scorer()

data_url = 'https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_testdata_CAT.json?raw=true'
r = requests.get(data_url)
test_data = r.json()

for text, entity_offsets in test_data:
    doc = nlp.make_doc(text)
    gold = GoldParse(doc, entities=entity_offsets.get('entities'))
    doc = nlp(text)
    scorer.score(doc, gold)
print('Precision: ',scorer.scores['ents_p'])
print('Recall: ',scorer.scores['ents_r'])
print('F1: ',scorer.scores['ents_f'])
#print(scorer.scores)

Are these good results do you think? Can this be improved? What happens if you increase the number of iterations in the training?

Let's look at a document from the test data.

In [ ]:
text, entity_offsets = test_data[37]
doc2 = nlp(text)
colors = {'ANATOMY': 'lightyellow',
           'DISEASESYNDROME': 'pink',  
           'SIGNSYMPTOM': 'lightgreen'}
displacy.render(doc2, style='ent', jupyter=True, options={'colors':colors})

What does the underlying representation look like?

In [ ]:
doc2 = nlp.make_doc(text)
gold = GoldParse(doc2, entities=entity_offsets.get('entities'))
gold.ner

# 4: Training a model with crfsuite
There are other machine learning algorithms that can be used for this sequence learning problem. Let's try crfsuite. If you don't have this package, install with: pip install sklearn-crfsuite

Let's use some functions to get sentences and tokens in the right format.

In [7]:
def get_sentences(filename, bio=False):
    
    r = requests.get(filename)
    train_data = r.json()

    sentences = []
    nlp = spacy.load('en', entity=False, parser=False)

    print('reading data: ', filename)
    for text, entity_offsets in train_data:
        doc = nlp(text)
        gold = spacy.gold.biluo_tags_from_offsets(doc, entity_offsets.get('entities'))
        counter = 0
        for s in doc.sents:
            sent = []
            for t in s:
                l = gold[counter]
                if bio:
                    l = l.replace('L-', 'I-')
                    l = l.replace('U-', 'B-')
                w = t.text, t.pos_, l
                sent.append(w)
                counter +=1
            sentences.append(sent)
    print('done')
    return sentences

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]



Let's use these functions and read in the training and test data. 
There are different alternative token level representations that can be used.
The BIO format (Begin, Inside, Outside) or the BILOU format (Begin, Inside, Last, Outside, Unit).
What do you think is better or worse with each of these?
In the function below, you can choose either format with the boolean flag 'bio'. Let's start with BIO.

In [9]:
train_sents = get_sentences('https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_trainingdata_CAT.json?raw=true', bio=True)
test_sents = get_sentences('https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_testdata_CAT.json?raw=true', bio=True)

reading data:  https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_trainingdata_CAT.json?raw=true
done
reading data:  https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_testdata_CAT.json?raw=true
done


Now let's create the feature and label vectors for the training and test data.

In [10]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

What labels do we have?

In [11]:
labels = list(set(x for l in y_test for x in l))
labels

['B-DISEASESYNDROME',
 'I-DISEASESYNDROME',
 'B-ANATOMY',
 '-',
 'B-SIGNSYMPTOM',
 'I-SIGNSYMPTOM',
 'I-ANATOMY',
 'O']

Now let's train the model.

In [12]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

# 5: evaluation
How does this model perform on this data?

In [13]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.9819657932011259

In [14]:
print(metrics.flat_classification_report(y_test, y_pred, labels = labels))

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                   precision    recall  f1-score   support

B-DISEASESYNDROME       0.89      0.73      0.80       489
I-DISEASESYNDROME       0.89      0.57      0.69       263
        B-ANATOMY       0.95      0.85      0.90       950
                -       0.00      0.00      0.00         1
    B-SIGNSYMPTOM       0.93      0.73      0.82       308
    I-SIGNSYMPTOM       0.92      0.69      0.79       125
        I-ANATOMY       0.92      0.80      0.85       363
                O       0.99      1.00      0.99     35888

        micro avg       0.98      0.98      0.98     38387
        macro avg       0.81      0.67      0.73     38387
     weighted avg       0.98      0.98      0.98     38387



What do you think? There's a huge imbalance in the number of instances. Do we really want to evaluate the 'O' label? There's also one instance with an erroneous label ('-') Let's look at the results without these labels.

In [15]:
labels = list(set(x for l in y_test for x in l if x !='O' and x!='-'))
labels

['B-DISEASESYNDROME',
 'I-DISEASESYNDROME',
 'B-ANATOMY',
 'B-SIGNSYMPTOM',
 'I-SIGNSYMPTOM',
 'I-ANATOMY']

In [16]:
print(metrics.flat_classification_report(y_test, y_pred, labels = labels))

                   precision    recall  f1-score   support

B-DISEASESYNDROME       0.89      0.73      0.80       489
I-DISEASESYNDROME       0.89      0.57      0.69       263
        B-ANATOMY       0.95      0.85      0.90       950
    B-SIGNSYMPTOM       0.93      0.73      0.82       308
    I-SIGNSYMPTOM       0.92      0.69      0.79       125
        I-ANATOMY       0.92      0.80      0.85       363

        micro avg       0.92      0.77      0.84      2498
        macro avg       0.92      0.73      0.81      2498
     weighted avg       0.92      0.77      0.84      2498



This was quite different! 
Try training this model with the BILOU scheme instead. Are results better or worse?

In [19]:
train_sents = get_sentences('https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_trainingdata_CAT.json?raw=true', bio=False)
test_sents = get_sentences('https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_testdata_CAT.json?raw=true', bio=False)
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

new_classes = list(set(x for l in y_test for x in l if x !='O' and x!='-'))

c1=0.1
c2=0.1

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1,
    c2=c2,
    max_iterations=100,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train)
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

reading data:  https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_trainingdata_CAT.json?raw=true
done
reading data:  https://github.com/KCL-Health-NLP/nlp_examples/blob/master/chunking/chunking_testdata_CAT.json?raw=true
done
                   precision    recall  f1-score   support

B-DISEASESYNDROME       0.86      0.63      0.73       190
I-DISEASESYNDROME       0.91      0.40      0.55        73
U-DISEASESYNDROME       0.90      0.79      0.84       299
        B-ANATOMY       0.88      0.76      0.81       277
    B-SIGNSYMPTOM       0.88      0.67      0.76        87
    L-SIGNSYMPTOM       0.92      0.70      0.80        87
        I-ANATOMY       0.97      0.74      0.84        84
        U-ANATOMY       0.97      0.87      0.92       673
    U-SIGNSYMPTOM       0.95      0.76      0.85       221
    I-SIGNSYMPTOM       0.89      0.66      0.76        38
L-DISEASESYNDROME       0.90      0.66      0.76       190
        L-ANATOMY       0.91      0.79 

# Optional: cross-validation to find best parameters with crfsuite
We have used default parameters in the above. We can try to find the best parameters on the training data by cross-validation. __This takes some time though (even with only 3 folds)!__ 

In [ ]:
# from: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#hyperparameter-optimization

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=new_classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=new_classes, digits=3
))

What do you think? Are there other parameters that could be tested in the cross-validation setup? What about the measure used for optimisation?